In [1]:
import pandas as pd
import pyodbc

conn_str = (
        "Driver={SQL Server};"
        "Server=DESKTOP-U9CSI9F\\SQLEXPRESS;"
        "Database=PortfolioProject_MarketingAnalytics;"
        "Trusted_Connection=yes;"
)

conn = pyodbc.connect(conn_str)
query = "SELECT ReviewID, CustomerID, ProductID, ReviewDate, Rating, ReviewText FROM dbo.customer_reviews"

df = pd.read_sql(query, conn)
conn.close()

C:\Users\user\AppData\Local\Temp\ipykernel_15368\3673997619.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [2]:
df.head()

,ReviewID,CustomerID,ProductID,ReviewDate,Rating,ReviewText
0,1,77,18,2023-12-23,3,"Average experience, nothing special."
1,2,80,19,2024-12-25,5,The quality is top-notch.
2,3,50,13,2025-01-26,4,Five stars for the quick delivery.
3,4,78,15,2025-04-21,3,"Good quality, but could be cheaper."
4,5,64,2,2023-07-16,3,"Average experience, nothing special."


In [3]:
df.dtypes

ReviewID       int64
CustomerID     int64
ProductID      int64
ReviewDate    object
Rating         int64
ReviewText    object
dtype: object

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1363 entries, 0 to 1362
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   ReviewID    1363 non-null   int64 
 1   CustomerID  1363 non-null   int64 
 2   ProductID   1363 non-null   int64 
 3   ReviewDate  1363 non-null   object
 4   Rating      1363 non-null   int64 
 5   ReviewText  1363 non-null   object
dtypes: int64(4), object(2)
memory usage: 64.0+ KB


In [5]:
df.describe()

,ReviewID,CustomerID,ProductID,Rating
count,1363.00000,1363.000000,1363.000000,1363.000000
mean,682.00000,51.701394,10.337491,3.686720
std,393.60852,28.605493,5.702922,1.180243
min,1.00000,1.000000,1.000000,1.000000
25%,341.50000,27.000000,5.000000,3.000000
50%,682.00000,53.000000,10.000000,4.000000
75%,1022.50000,77.000000,15.000000,5.000000
max,1363.00000,100.000000,20.000000,5.000000


In [6]:
df.duplicated().sum()

0

In [7]:
df.isnull().sum()

ReviewID      0
CustomerID    0
ProductID     0
ReviewDate    0
Rating        0
ReviewText    0
dtype: int64

In [8]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [9]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sia = SentimentIntensityAnalyzer()


#Define a function to categorize sentiment using both the sentiment score and the review rating

In [10]:
df.head()

,ReviewID,CustomerID,ProductID,ReviewDate,Rating,ReviewText
0,1,77,18,2023-12-23,3,"Average experience, nothing special."
1,2,80,19,2024-12-25,5,The quality is top-notch.
2,3,50,13,2025-01-26,4,Five stars for the quick delivery.
3,4,78,15,2025-04-21,3,"Good quality, but could be cheaper."
4,5,64,2,2023-07-16,3,"Average experience, nothing special."


In [11]:
# 1. Calculate sentiment score
def calculate_sentiment(review):
    sentiment = sia.polarity_scores(str(review))
    return sentiment['compound']

In [12]:
# 2. Categorize sentiment using score + rating
def categorize_sentiment(score, rating):
    if score > 0.05:
        if rating >= 4:
            return 'Positive'
        elif rating == 3:
            return 'Mixed Positive'
        else:
            return 'Mixed Negative'
    elif score < -0.05:
        if rating <= 2:
            return 'Negative'
        elif rating == 3:
            return 'Mixed Negative'
        else:
            return 'Mixed Positive'
    else:
        if rating >= 4:
            return 'Positive'
        elif rating <= 2:
            return 'Negative'
        else:
            return 'Neutral'


In [13]:
# 3. Bucket sentiment score
def sentiment_bucket(score):
    if score >= 0.5:
        return '0.5 to 1.0'
    elif 0.0 <= score < 0.5:
        return '0.0 to 0.49'
    elif -0.5 <= score < 0.0:
        return '-0.49 to 0.0'
    else:
        return '-1.0 to -0.5'


In [14]:
df['SentimentScore'] = df['ReviewText'].apply(calculate_sentiment)
df['SentimentCategory'] = df.apply(lambda row: categorize_sentiment(row['SentimentScore'], row['Rating']), axis=1)
df['SentimentBucket'] = df['SentimentScore'].apply(sentiment_bucket)

# Show the results
df.head()


,ReviewID,CustomerID,ProductID,ReviewDate,Rating,ReviewText,SentimentScore,SentimentCategory,SentimentBucket
0,1,77,18,2023-12-23,3,"Average experience, nothing special.",-0.3089,Mixed Negative,-0.49 to 0.0
1,2,80,19,2024-12-25,5,The quality is top-notch.,0.0000,Positive,0.0 to 0.49
2,3,50,13,2025-01-26,4,Five stars for the quick delivery.,0.0000,Positive,0.0 to 0.49
3,4,78,15,2025-04-21,3,"Good quality, but could be cheaper.",0.2382,Mixed Positive,0.0 to 0.49
4,5,64,2,2023-07-16,3,"Average experience, nothing special.",-0.3089,Mixed Negative,-0.49 to 0.0


In [15]:
#df.to_csv('customer_reviews_sentiment.csv', index=False)